In [ ]:
# default_exp deepzoom

In [ ]:
#hide 
%load_ext autoreload
%autoreload 2

# Functions 

In [ ]:
#export 
import pyvips 
import os 
import shutil 
import glob
import re 
from bs4 import BeautifulSoup 
import sys
import time 
import b2sdk.v1 as b2
import numpy as np
import json 
import requests 

ROOTDIR = 'deepzoom'
VIEWERSDIR = 'viewers'

class DeepZoomStore: 
    
    def __init__(self, application_key_id, application_key, bucket_name): 
        ''''''
        
        # create local directory structure in current working directory 
        store_dir = os.path.join('.', ROOTDIR)
        os.makedirs(store_dir, exist_ok=True) 
        
        viewers_dir = os.path.join('.', ROOTDIR, VIEWERSDIR)
        os.makedirs(viewers_dir, exist_ok=True) 
        
        # connect to b2 bucket 
        self.application_key_id = application_key_id 
        self.application_key = application_key
        self.bucket_name = bucket_name 
        
        self.b2_api, self.base_url = connect_b2_bucket(application_key_id, application_key, bucket_name)
        
        # initialize image list 
        self.images = []

        
    def make_deepzoom(self, fname): 
         
        # convert to local deepzoom directory 
        name = dzp_save(fname)
        self.images.append(name) 
        
        # also make thumnail 
        dzp_thumbnail(fname)
        
        dzi_to_js(name, self.base_url, self.bucket_name)
        make_html(name, self.bucket_name) 
        
        
        # bit rough but just update for every added image 
        viewer_name = 'overview'
        
        self.create_multiviewer(viewer_name, selection='all')

            
        # and sync new files to b2 bucket
        upload(self.b2_api, self.bucket_name) 
        

        
        
    def create_multiviewer(self, viewer_name, selection='all'): 
        
        print('Automatically updating b2 bucket to include all newly created images...') 
        upload(self.b2_api, self.bucket_name) 
        
        make_multiviewer_jsonp(viewer_name, self.application_key_id, 
                               self.application_key, self.bucket_name, 
                               selection=selection)
        
        # hard coded script src, need to make dynamic! But not today.  
        make_multiviewer_html(viewer_name)
        
        print(f'Automatically uploading multiviewer files for: {viewer_name}...')
        
        upload(self.b2_api, self.bucket_name) 
        
    def list_names(self):
        
        names, js_urls = list_names_and_js_urls(self.application_key_id, self.application_key, self.bucket_name) 
        
        return names 
             
                

def upload(b2_api, bucket_name):
    ''''''

    source = os.path.join('.', ROOTDIR)
    destination = f'b2://{bucket_name}/{ROOTDIR}'  

    source = b2.parse_sync_folder(source, b2_api)
    destination = b2.parse_sync_folder(destination, b2_api)

    policies_manager = b2.ScanPoliciesManager(exclude_all_symlinks=True)

    synchronizer = b2.Synchronizer(
            max_workers=10,
            policies_manager=policies_manager,
            dry_run=False,
            allow_empty_source=True,
        )

    no_progress = False 

    # need to run this to actually synchronize 

    with b2.SyncReport(sys.stdout, no_progress) as reporter:
            synchronizer.sync_folders(
                source_folder=source,
                dest_folder=destination,
                now_millis=int(round(time.time() * 1000)),
                reporter=reporter,
            ) 
            
        
    

def make_html(name, bucket_name): 
    
    html_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}_view.html') 
    
    if not os.path.exists(html_fname):
    
        html = f'''
<!DOCTYPE html>
<html>
<body style="width: 800px; height: 600px; background-color: snow">

<div id="openseadragon1" style="width: 100%; height: 100%; background-color: gray"></div> 

<script src="https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/openseadragon.min.js" 
        integrity="sha512-qvQYH6mPuE46uFcWLI8BdGaJpB5taX4lltbSIw5GF4iODh2xIgyz5ii1WpuzPFUknHCps0mi4mFGR44bjdZlZg==" 
        crossorigin="anonymous">
</script>

<script type="text/javascript">
var viewer = OpenSeadragon({{
    id: "openseadragon1",
    prefixUrl: "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/images/", 
    tileSources: "{name}.dzi", 
 }});
</script>


</body>
</html>  
'''
        with open(html_fname, 'w') as fh: 

            print(f'Saving: {html_fname}')
            fh.writelines(html)
            
    else: 
        print(f'Not overwriting existing files: {html_fname}')
       

    

def connect_b2_bucket(application_key_id, application_key, bucket_name): 
    
    '''Creates upload connection to your Backblaze b2 bucket.'''
    
    info = b2.InMemoryAccountInfo()
    b2_api = b2.B2Api(info) 
    b2_api.authorize_account("production", application_key_id, application_key)  
    
    
    bucket = b2_api.get_bucket_by_name(bucket_name) 

    filenames = [file_info.file_name for file_info, _ in bucket.ls(show_versions=False, recursive=False)]
    urls = [b2_api.get_download_url_for_file_name(bucket_name, f) for f in filenames] 
    base_url = re.split(bucket_name, urls[0])[0] 
    
    return b2_api, base_url


def dzp_save(fname): 
    # name = dzp_save(fname, report_tree=report_tree) 
    
    name, ext = os.path.splitext(os.path.basename(fname))
    dzp_fname = os.path.join('.', ROOTDIR, f'dzp_{name}')
    dzi_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}.dzi') 
    dzi_base = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}')
    
    os.makedirs(dzp_fname, exist_ok=True) 
    
    if not os.path.exists(dzi_fname): 
        v_img = pyvips.Image.new_from_file(fname) 
        v_img.dzsave(dzi_base)
    else: 
        print(f'Not overwriting existing files: {dzi_base}')
    
    return name 


    
    
def dzi_to_js(name, base_url, bucket_name): 
    '''Parse XML.dzi file into JSONP.js file. 
    
    '''
    
    url = f'{base_url}{bucket_name}/{ROOTDIR}/dzp_{name}/{name}_files/'  
    dzi_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}.dzi') 
    jsonp_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}.js') 
    
    if not os.path.exists(jsonp_fname):
    
        with open(dzi_fname, 'r') as fh: 
            lines = fh.read()

        soup = BeautifulSoup(lines, 'xml')

        Format = soup.Image.attrs['Format']
        Overlap = soup.Image.attrs['Overlap']
        TileSize = soup.Image.attrs['TileSize']
        Height = soup.Image.Size.attrs['Height'] 
        Width = soup.Image.Size.attrs['Width']

        jsonp = f'''OpenSeadragon(\
{{
"id": "{name}", 
"prefixUrl": "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/images/", 
"tileSources": {{
    "Image": {{
       "xmlns":    "http://schemas.microsoft.com/deepzoom/2008",
       "Url":      "{url}", 
       "Format": "{Format}", 
       "Overlap": "{Overlap}",  
       "TileSize": "{TileSize}", 
       "Size": {{
           "Height": "{Height}", 
           "Width": "{Width}" 
        }}
    }}
}}
}}); 
'''

        with open(jsonp_fname, 'w') as fh: 
            print(f'Saving: {jsonp_fname}')
            fh.writelines(jsonp)
            
    else: 
        print(f'Not overwriting existing file: {jsonp_fname}')


def list_names_and_js_urls(application_key_id, application_key, bucket_name):  
    
    # connect 
    b2_api, base_url =connect_b2_bucket(application_key_id, application_key, bucket_name)
    bucket = b2_api.get_bucket_by_name(bucket_name) 
    
    # list filenames in deepzoom bucket folder 
    filenames = [file_info.file_name for file_info, _ in 
                 bucket.ls(folder_to_list=ROOTDIR, 
                           show_versions=False, recursive=False)] 
    
    # extract names 
    ptrn = f'{ROOTDIR}/dzp_(?P<name>[^/]+).*'

    names = []
    for f in filenames: 
        m = re.match(ptrn, f)
        if m: 
            names.append(m.groups()[0])
            
    # infer js urls 
    js_urls = [f'{base_url}{bucket_name}/{ROOTDIR}/dzp_{n}/{n}.js' for n in names]
    
    return names, js_urls

def stack_image_json(js_urls): 
    
    # download and parse .js files and append into multi tileSources string 
    jsonp_strings = []

    ptrn = '.*"tileSources":(?P<tsources>.*)\s*\}\s*\);' 
    repl = '\g<tsources>'

    for u in js_urls: 
        # download jsonp.js file 
        s = requests.get(u).content.decode() 
        # extract tileSources substring 
        s = re.sub(ptrn, repl, s, flags=re.DOTALL)
        # check valid json and improve formatting
        s = json.dumps(json.loads(s), indent=2)

        jsonp_strings.append(s)

    multi_tileSources = ',\n'.join(jsonp_strings)
    
    return multi_tileSources 




def make_multiviewer_jsonp(viewer_name, application_key_id, application_key, bucket_name, selection='all'): 
    '''Download all single image JSONP.js files and combine in multi-viewer.js '''
    
    # connect to bucket and list all deep zoom images 
    names, js_urls = list_names_and_js_urls(application_key_id, application_key, bucket_name) 
    
    # limit js_urls to selection   
    if selection is not 'all': 
        indices =  []
        for s in selection: 
            if s in names: 
                indices.append(names.index(s)) 
            else: 
                print(f'Warning: Could not find image: {s}')
                     
        js_urls = np.array(js_urls)[indices]
           
    # download jsonp and extract tile sources json
    multi_tileSources = stack_image_json(js_urls)

    
    
    # add openseadragon callback and viewer settings 
    multi_jsonp = f'''OpenSeadragon( \
{{
"id": "{viewer_name}", 
"prefixUrl": "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/images/", 
"tileSources": [{multi_tileSources}], 
"sequenceMode": "true", 
"preserveViewport": "false", 
"showReferenceStrip": "true"
}}); 
'''
    
    # write to local multiviewer.js file  
    multiviewer_fname = os.path.join('.', ROOTDIR, VIEWERSDIR, f'{viewer_name}.js')
    
    with open(multiviewer_fname, 'w') as fh: 
        print(f'Saving: {multiviewer_fname}')
        fh.writelines(multi_jsonp)

        
def make_multiviewer_html(viewer_name): 
    
        
    # need to make src attribute dynamic 
    html_multiviewer_fname = os.path.join('.', ROOTDIR, VIEWERSDIR, f'{viewer_name}.html') 
    
    #if not os.path.exists(html_multiviewer_fname):
    
    html = f'''
<!DOCTYPE html>
<html>
<body style="width: 800px; height: 600px; background-color: snow">

<script src="https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/openseadragon.min.js" 
        integrity="sha512-qvQYH6mPuE46uFcWLI8BdGaJpB5taX4lltbSIw5GF4iODh2xIgyz5ii1WpuzPFUknHCps0mi4mFGR44bjdZlZg==" 
        crossorigin="anonymous">
</script>
<div id="{viewer_name}" style="width: 800px; height: 400px; background-color: snow"></div>

<script src="https://f002.backblazeb2.com/file/dore-data/deepzoom/viewers/{viewer_name}.js?callback=OpenSeadragon"></script>

</body>
</html>  
'''
    with open(html_multiviewer_fname, 'w') as fh: 

        print(f'Saving: {html_multiviewer_fname}')
        fh.writelines(html)
            
    #else: 
    #    print(f'Not overwriting existing files: {html_multiviewer_fname}')


def dzp_thumbnail(fname, tn_height=500): 
    '''Creates thumbnail file in dzp_ directory'''
    
    # create directory if needed 
    name, ext = os.path.splitext(os.path.basename(fname))
    dzp_fname = os.path.join('.', ROOTDIR, f'dzp_{name}')
    os.makedirs(dzp_fname, exist_ok=True)

    # construct standard thumbnail filename 
    thumb_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'tn_{name}.png') 
    
    # resize 
    tn_height = 500 
    v_img = pyvips.Image.new_from_file(fname) 
    scale = tn_height / v_img.height
    v_thumb = v_img.resize(scale)
    
    # write to file 
    v_thumb.write_to_file(thumb_fname)

## Usage  

Will be useful to include thumbnails in the dzp_ folders to create overviews. 

In [ ]:
#hide
import os 
import myb2keys 
import deepzoomup 

os.chdir('/home/frank/Work/Projecten/DoRe/viz') 

application_key = myb2keys.application_key 
application_key_id = myb2keys.application_key_id 
bucket_name = myb2keys.bucket_name 

dz = deepzoomup.DeepZoomStore(application_key_id, application_key, bucket_name)

In [ ]:
#hide 
fname1 = '/home/frank/Work/Projecten/DoRe/data/drawings/rma-web-highres/RP-T-1930-22_highres.png' 
fname2 = '/home/frank/Work/Projecten/DoRe/data/drawings/rma-web-highres/RP-T-1930-51_highres.png'

In [ ]:
dz.make_deepzoom(fname2)

Not overwriting existing files: ./deepzoom/dzp_RP-T-1930-51_highres/RP-T-1930-51_highres
Not overwriting existing file: ./deepzoom/dzp_RP-T-1930-51_highres/RP-T-1930-51_highres.js
Not overwriting existing files: ./deepzoom/dzp_RP-T-1930-51_highres/RP-T-1930-51_highres_view.html
Automatically updating b2 bucket to include all newly created images...
upload dzp_RP-T-1930-51_highres/tn_RP-T-1930-51_highres.png           
Saving: ./deepzoom/viewers/overview.js                           
Saving: ./deepzoom/viewers/overview.html
Automatically uploading multiviewer files for: overview...
upload viewers/overview.js                                               
upload viewers/overview.html                                             


## Thumbnails x

In [ ]:
import pyvips 
import os 

ROOTDIR = 'deepzoom'

In [ ]:
def dzp_thumbnail(fname, tn_height=500): 
    '''Creates thumbnail file in dzp_ directory'''
    
    # create directory if needed 
    name, ext = os.path.splitext(os.path.basename(fname))
    dzp_fname = os.path.join('.', ROOTDIR, f'dzp_{name}')
    os.makedirs(dzp_fname, exist_ok=True)

    # construct standard thumbnail filename 
    thumb_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'tn_{name}.png') 
    
    # resize 
    tn_height = 500 
    v_img = pyvips.Image.new_from_file(fname) 
    scale = tn_height / v_img.height
    v_thumb = v_img.resize(scale)
    
    # write to file 
    v_thumb.write_to_file(thumb_fname)

In [ ]:
dzp_thumbnail(fname2)

In [ ]:
!ls

curtain_viewers  deepzoom  ma-xrf-viz  thumb.png


In [ ]:
v_thumb = v_img.thumbnail_buffer(width=500, height=500)

Error: resize needs 2 arguments, but 1 given
  

In [ ]:
v_thumb

<pyvips.Image 2218x2514 uchar, 4 bands, srgb>

In [ ]:
v_thumb = v_img.vips_thumbnail(500)

Error: no such operation vips_thumbnail
  VipsOperation: class "vips_thumbnail" not found


In [ ]:
!ls

curtain_viewers  deepzoom  ma-xrf-viz


In [ ]:
v_img.resize(500)

<pyvips.Image 2218000x2514000 uchar, 4 bands, srgb>

In [ ]:
v_img.resize(w=500, h=500)

Error: resize needs 2 arguments, but 1 given
  

In [ ]:
v_img.write_to_file('tn_maria.png', 500)

TypeError: write_to_file() takes 2 positional arguments but 3 were given

## Listing uploaded images 

To list all image names: 

In [ ]:
dz.list_names()

['RP-T-1930-22_highres', 'RP-T-1930-51_highres', 'RP-T-1930-9_Trans-Recto']

In [ ]:
selected = ['RP-T-1930-22_highres', 'RP-T-1930-51_highres']

In [ ]:
#hide 
fname1 = '/home/frank/Work/Projecten/DoRe/data/drawings/rma-web-highres/RP-T-1930-22_highres.png' 
fname2 = '/home/frank/Work/Projecten/DoRe/data/drawings/rma-web-highres/RP-T-1930-51_highres.png'

dz.make_deepzoom(fname2);

Not overwriting existing files: ./deepzoom/dzp_RP-T-1930-51_highres/RP-T-1930-51_highres
Not overwriting existing file: ./deepzoom/dzp_RP-T-1930-51_highres/RP-T-1930-51_highres.js
Not overwriting existing files: ./deepzoom/dzp_RP-T-1930-51_highres/RP-T-1930-51_highres_view.html
Automatically updating b2 bucket to include all newly created images...
Saving: ./deepzoom/viewers/overview.js                             
Saving: ./deepzoom/viewers/overview.html
Automatically uploading multiviewer files for: overview...
upload viewers/overview.html                                           
upload viewers/overview.js                                             


In [ ]:
#hide
fname3 = '/media/frank/Backup Plus/Rembrandt tekeningen/RP-T-1930-9/RP-T-1930-9_tif/RP-T-1930-9_Trans-Recto.tif'

dz.make_deepzoom(fname3)

Saving: ./deepzoom/dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto.js
Saving: ./deepzoom/dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_view.html
Automatically updating b2 bucket to include all newly created images...
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/10/0_0.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/10/1_0.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/10/2_0.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/10/1_1.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/10/1_2.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/10/0_2.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/10/0_1.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto.js     
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/11/0_0.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/11/0_1.jpeg
upload dzp_RP

upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_13.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_17.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_4.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_14.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_5.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_7.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_9.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_2.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_12.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/13_0.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/12_6.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/13_1.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_f

upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/22_6.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/2_1.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/22_9.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/2_13.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/2_11.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/2_14.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/22_8.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/2_16.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/2_15.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/21_3.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/2_10.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/13/2_2.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/1

upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/10_6.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/10_7.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/10_31.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/11_0.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/10_9.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/11_1.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/10_8.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/11_13.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/10_33.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/11_12.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/11_11.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/11_14.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto

upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/16_27.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/16_3.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/16_4.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/16_8.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/16_34.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/16_7.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/16_32.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/16_6.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/17_1.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/17_10.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/17_11.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/14/17_12.jpeg
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto

...

upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_19.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_3.jpeg    
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_16.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_28.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_30.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_33.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_39.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_38.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_35.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_31.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_4.jpeg    
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/80_43.jpeg   
upload dzp_RP-T-

upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_21.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_14.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_22.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_24.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_25.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_20.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_26.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_23.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_29.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_12.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_3.jpeg    
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/83_31.jpeg   
upload dzp_RP-T-

upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/85_8.jpeg    
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_15.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_16.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/85_30.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_17.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_13.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_2.jpeg    
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_14.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_19.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_11.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_22.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/86_25.jpeg   
upload dzp_RP-T-

upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/88_65.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/88_8.jpeg    
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/88_59.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/88_68.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/88_54.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/89_0.jpeg    
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/88_67.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/89_10.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/89_13.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/89_1.jpeg    
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/89_17.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/89_11.jpeg   
upload dzp_RP-T-

upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_43.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_55.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_62.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_50.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_63.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_66.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_65.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_59.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_68.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_7.jpeg    
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/90_61.jpeg   
upload dzp_RP-T-1930-9_Trans-Recto/RP-T-1930-9_Trans-Recto_files/15/89_44.jpeg   
upload dzp_RP-T-